In [ ]:
import pyspark
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
import matplotlib.pyplot as plt
import pyspark.pandas as ps

spark = SparkSession.builder.appName("bike").getOrCreate()
df = spark.read.option("header","true").csv("2017-fordgobike-tripdata.csv")

In [ ]:
from haversine import haversine, Unit

haversine([45.7597, 4.8422], [48.8567, 2.3508])

In [ ]:
def getDistance(start_lat, start_long, end_lat, end_long):
    return round(
        haversine(
            [ float(start_lat), float(start_long) ],
            [ float(end_lat), float(end_long) ],
            unit=Unit.METERS
        ), 2
    )

In [ ]:
getDistanceUDF = udf(lambda a,b,c,d : getDistance(a,b,c,d), IntegerType() )
df = df.withColumn("Distance", getDistanceUDF(df["start_station_latitude"], df["start_station_longitude"], df["end_station_latitude"], df["end_station_longitude"]))

# Task 4

In [ ]:
df = df.withColumn("age", year(current_date()) - year(col("member_birth_year")))

In [ ]:
df = df.dropna(subset=["age", "distance"])

In [ ]:
df_p = df.select(["age", "distance"])
df_pt = df_p.toPandas()
plt.scatter(df_pt["age"], df_pt["distance"], s=30)
plt.xlabel("age")
plt.ylabel("distance")
plt.title("Age vs Distance")
plt.show()